In [ ]:
import gym # Подключаем игровой модуль gym
import random # Подключаем модель генерации случайных чисел random
import numpy as np # Подключаем модуль numpy
from collections import deque # Подключаем класс deque (очередь) модуля collections
from skimage.color import rgb2gray # Подключаем метод rgb2gray (РГБ в градации серого) библиотеки skimage.color
from skimage.transform import resize # Подключаем метод resize библиотеки skimage.transform
from keras.models import Sequential # Подключаем класс Sequential библиотеки keras.models
from keras.optimizers import RMSprop # Подключаем оптимизатор RMSprop библиотеки keras.optimizers
from keras.layers import Dense, Flatten # Подключаем слои Dense, Flatten библиотеки keras.layers
from keras.layers.convolutional import Conv2D # Подключаем слой Conv2D библиотеки keras.layers.convolutional
from keras import backend as K # Подключаем модуль backend библиотеки keras

In [ ]:
EPISODES = 50000 # Устанавливаем максимальное количество эпизодов

In [ ]:
# Создаем класс агента
class Agent:
    # Инициализация
    def __init__(self, action_size, epsilon):
        self.render = False # Требуется ли рендерить игровой процесс
        self.load_model = False # Требуется ли загружать предобученную модель
        
        self.state_size = (84, 84, 4) # Устанавиваем размеры игрового кадра
        self.action_size = action_size # Получаем количество возможных действий агента
        
        self.epsilon = epsilon # Устанавливаем текущее значение epsilon
        self.epsilon_start, self.epsilon_end = 1.0, 0.1 # Устанавливаем начальное и конечное значение epsilon
        self.exploration_steps = 1000000. # Устанавливаем количество шагов, во время которых будет изменять epsilon
        self.epsilon_decay_step = (self.epsilon_start - self.epsilon_end) \
                                  / self.exploration_steps # Устанавливаем шаг изменения epsilon
        
        self.batch_size = 32 # Устанавливаем размер батча
        self.train_start = 50000 # Устанавливаем количество шагов, после которых начинаем обучение модели        
        self.update_target_rate = 10000 # Устанавливаем количество шагов, после которых обновляем веса второй модели
        self.discount_factor = 0.99 # Устанавливаем коэффициент снижения весов
        self.memory = deque(maxlen=400000) # Создаем объект deque, указывая макстмальное количество элементов
        self.no_op_steps = 30 # Устанавливаем количество шагов, в течении которых вначале эпизода ничего не делаем
        
        self.model = self.build_model() # Создаем основную модель
        self.target_model = self.build_model() # Создаем дублирующую модель
        self.update_target_model() # Обновляем веса дублирующей модели

        self.avg_q_max, self.avg_loss = 0, 0 # Инициализируем среднее вознаграждении и среднюю ошибку
        
        if self.load_model: # Если требуется загрузить веса
            self.model.load_weights("model/breakout_dqn.h5") # Загружаем предобученные веса
    
    # Функция ошибки
    def q_loss(self, y_true, y_pred):
        error = K.abs(y_true - y_pred) # Плучаем модуль разности между предиктом и реальным значением
        q_part = K.clip(error, 0.0, 1.0) # Приводим ошибку error в интервал от 0 до 1
        linear_part = error - q_part # Получаем разницу между ошибкой error и ее clip'ом (все, что вышло за единицу)
        loss = K.mean(0.5 * K.square(q_part) + linear_part) # Считаем среднеквадратичную ошибку и выход за значение единицы
        return loss # Возвращаем loss

    # Строим модель
    def build_model(self):
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu',
                         input_shape=self.state_size))
        model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
        model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size))
        model.summary()
        model.compile(optimizer = RMSprop(lr=0.00025, epsilon=0.01), loss = self.q_loss)
        return model
        
    # Функция обновления весов дублирующей модели
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights()) # Устанавливаем веса дублирующей модели с весов основной

    # Функция получения действия агента
    def get_action(self, history):
        history = np.float32(history / 255.0) # Делим кадр на 255 (нормируем значения)
        if np.random.rand() <= self.epsilon: # Если случайно-сгенерированное число от 0 до 1 меньше текущего epsilon
            return random.randrange(self.action_size) # Возвращаем случайное действие из возможных от 0 до action_size
        else:
            q_value = self.model.predict(history) # Получаем предикт модели по текущему кадру
            return np.argmax(q_value[0]) # Возвращаем индекс максимального вознаграждения

    # Функция записи в память 
    def replay_memory(self, history, action, reward, next_history, dead):
        # Записываем в памть текущий кадр, совершенное действие, полученную награду, следующий кадр и признак завершения партии
        self.memory.append((history, action, reward, next_history, dead)) 

    # Функция обучения модели
    def train_replay(self):
        if len(self.memory) < self.train_start: # Если текущий размер памяти агента меньше установленного train_start
            return # Выходим и не обучаем модель
        if self.epsilon > self.epsilon_end: # Если значение epsilon меньше epsilon_end
            self.epsilon -= self.epsilon_decay_step # меньшаем значение epsilon на epsilon_decay_step

        mini_batch = random.sample(self.memory, self.batch_size) # Получаем из памяти агента случайный батч размером batch_size
        
        # Создаем numpy массив из batch_size-элементов с размерами игрового кадра
        history = np.zeros((self.batch_size, self.state_size[0],
                            self.state_size[1], self.state_size[2]))
        # Создаем numpy массив из batch_size-элементов с размерами игрового кадра
        next_history = np.zeros((self.batch_size, self.state_size[0],
                                 self.state_size[1], self.state_size[2]))
        target = np.zeros((self.batch_size,)) # Создаем numpy массив из batch_size-элементов
        
        action, reward, dead = [], [], [] # Инициализируем списки под действия, награды и признак завершения партии

        for i in range(self.batch_size):
            history[i] = np.float32(mini_batch[i][0] / 255.)
            next_history[i] = np.float32(mini_batch[i][3] / 255.)
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            dead.append(mini_batch[i][4])

        target_value = self.target_model.predict(next_history)
        q_value = self.model.predict(history)

        # like Q Learning, get maximum Q value at s'
        # But from target model
        for i in range(self.batch_size):
            if dead[i]:
                target[i] = reward[i]
            else:
                target[i] = reward[i] + self.discount_factor * \
                                        np.amax(target_value[i])
            q_value[i][action[i]] = target[i]
        #loss = self.optimizer([history, action, target])
        loss = self.model.train_on_batch(history, q_value)
        #self.avg_loss += loss[0]
        self.avg_loss += loss

    def save_model(self, name):
        self.model.save_weights(name)

In [ ]:
# 210*160*3(color) --> 84*84(mono)
# float --> integer (to reduce the size of replay memory)
def pre_processing(observe):
    processed_observe = np.uint8(
        resize(rgb2gray(observe), (84, 84), mode='constant') * 255)
    return processed_observe

In [ ]:
# In case of BreakoutDeterministic-v3, always skip 4 frames
# Deterministic-v4 version use 4 actions
env = gym.make('BreakoutDeterministic-v4')
agent = Agent(action_size=3, epsilon = 1)

scores, episodes, global_step = [], [], 0

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 1539      
Total params: 1,685,667
Trainable params: 1,685,667
Non-trainable params: 0
____________________________________________

In [ ]:
scores = []
average_scores = []
for e in range(1, EPISODES):
    done = False
    dead = False
    # 1 episode = 5 lives
    step, score, start_life = 0, 0, 5
    observe = env.reset()

    # this is one of DeepMind's idea.
    # just do nothing at the start of episode to avoid sub-optimal
    for _ in range(random.randint(1, agent.no_op_steps)):
        observe, _, _, _ = env.step(1)

    # At start of episode, there is no preceding frame
    # So just copy initial states to make history
    state = pre_processing(observe)
    history = np.stack((state, state, state, state), axis=2)
    history = np.reshape([history], (1, 84, 84, 4))

    while not done:
        if agent.render:
            env.render()
        global_step += 1
        step += 1

        # get action for the current history and go one step in environment
        action = agent.get_action(history)
        # change action to real_action
        if action == 0:
            real_action = 1
        elif action == 1:
            real_action = 2
        else:
            real_action = 3

        observe, reward, done, info = env.step(real_action)
        next_state = pre_processing(observe)
        next_state = np.reshape([next_state], (1, 84, 84, 1))
        next_history = np.append(next_state, history[:, :, :, :3], axis=3)

        agent.avg_q_max += np.amax(
            agent.model.predict(np.float32(history / 255.))[0])

        if start_life > info['ale.lives']:
            dead = True
            start_life = info['ale.lives']

        reward = np.clip(reward, -1., 1.)

        agent.replay_memory(history, action, reward, next_history, dead)
        agent.train_replay()
        if global_step % agent.update_target_rate == 0:
            agent.update_target_model()

        score += reward

        if dead:
            dead = False
        else:
            history = next_history

        if done:            
            if global_step > agent.train_start:                
                stats = [score, agent.avg_q_max / float(step), step,
                          agent.avg_loss / float(step)]
            scores.append(score)
            average_scores.append(agent.avg_q_max / float(step))
            print("ep:", e, "  s.:", int(score), "  mem.len:",
                  len(agent.memory), "  eps:", round(agent.epsilon,2),
                  "  g_step:", global_step, "  av_q:",
                  round(agent.avg_q_max / float(step),3), "  av_loss:",
                  round(agent.avg_loss / float(step),7))

            agent.avg_q_max, agent.avg_loss = 0, 0

    if e % 500 == 0:
        agent.model.save_weights("breakout_dqn.h5")
        np.save('scores', np.array(scores))
        np.save('average_scores', np.array(average_scores))

ep: 1   s.: 2   mem.len: 184   eps: 1   g_step: 184   av_q: 0.012   av_loss: 0.0
ep: 2   s.: 2   mem.len: 360   eps: 1   g_step: 360   av_q: 0.015   av_loss: 0.0
ep: 3   s.: 0   mem.len: 482   eps: 1   g_step: 482   av_q: 0.013   av_loss: 0.0
ep: 4   s.: 0   mem.len: 598   eps: 1   g_step: 598   av_q: 0.013   av_loss: 0.0
ep: 5   s.: 1   mem.len: 771   eps: 1   g_step: 771   av_q: 0.012   av_loss: 0.0
ep: 6   s.: 0   mem.len: 887   eps: 1   g_step: 887   av_q: 0.012   av_loss: 0.0
ep: 7   s.: 1   mem.len: 1042   eps: 1   g_step: 1042   av_q: 0.012   av_loss: 0.0
ep: 8   s.: 1   mem.len: 1211   eps: 1   g_step: 1211   av_q: 0.012   av_loss: 0.0
ep: 9   s.: 0   mem.len: 1324   eps: 1   g_step: 1324   av_q: 0.013   av_loss: 0.0
ep: 10   s.: 0   mem.len: 1425   eps: 1   g_step: 1425   av_q: 0.013   av_loss: 0.0
ep: 11   s.: 3   mem.len: 1657   eps: 1   g_step: 1657   av_q: 0.015   av_loss: 0.0
ep: 12   s.: 0   mem.len: 1757   eps: 1   g_step: 1757   av_q: 0.013   av_loss: 0.0
ep: 13   s.: 